In [1]:
import os
print("Working dir:", os.getcwd())


Working dir: /Users/user/Documents/DataEngineering


In [2]:
import pandas as pd

src = "SourceDatasets/(USE THIS)ad_events_header_updated.csv"
dst = "SourceDatasets/ad_events_new.csv"

df = pd.read_csv(src, nrows=10_000)
df.to_csv(dst, index=False)
df.head()


,EventID,AdvertiserName,CampaignName,CampaignStartDate,CampaignEndDate,CampaignTargetingCriteria,CampaignTargetingInterest,CampaignTargetingCountry,AdSlotSize,UserID,Device,Location,Timestamp,BidAmount,AdCost,WasClicked,ClickTimestamp,AdRevenue,Budget,RemainingBudget
0,e067dae7-9aaa-4b9c-9195-7af63ce89216,Advertiser_30,Campaign_278,2024-10-10,2024-11-12,Age 29-46,Gaming,Germany,300x250,289903,Mobile,USA,2024-10-31T06:56:39,4.60,3.70,False,NaN,0.0,279306.60,279306.60
1,ea95edfa-1ba0-444d-a40d-a3f064dd5791,Advertiser_25,Campaign_235,2024-10-24,2024-11-28,Age 22-33,Gaming,Australia,728x90,145276,Desktop,UK,2024-11-13T09:19:34,3.50,2.85,False,NaN,0.0,88914.60,88914.60
2,5f12c537-ac88-4fcc-8d23-b3fa2b538a3b,Advertiser_95,Campaign_947,2024-10-31,2024-11-11,Age 27-42,Health,USA,300x250,83608,Desktop,USA,2024-11-08T14:36:14,3.75,3.16,False,NaN,0.0,207892.45,207892.45
3,191d7957-90ec-4e6c-aaec-f70c6ca57513,Advertiser_53,Campaign_530,2024-10-25,2024-11-20,Age 18-36,Fashion,USA,300x250,280085,Mobile,Australia,2024-11-10T22:39:42,0.83,0.73,False,NaN,0.0,181597.78,181597.78
4,42a66a39-a275-4007-81de-21182d3ead70,Advertiser_5,Campaign_35,2024-10-02,2024-11-26,Age 30-43,Technology,India,728x90,656312,Mobile,USA,2024-10-17T20:05:20,0.91,0.82,False,NaN,0.0,354525.31,354525.31


In [6]:
import pandas as pd


SRC = "SourceDatasets/(USE THIS)ad_events_header_updated.csv"
DST = "SourceDatasets/ad_events_new.csv"

# 1) Зібрати унікальні місяці за Timestamp
months = set()
for chunk in pd.read_csv(
    SRC,
    usecols=["Timestamp"],
    parse_dates=["Timestamp"],
    chunksize=200_000
):
    months.update(chunk["Timestamp"].dt.to_period("M").unique())
months = sorted(months)
print("Місяці в даних:", months)

# 2) Задаємо квоту вручну
quota = 70_000
print(f"Квота на місяць: {quota} записів")

# 3) Проходимо по подіях чанками та збираємо
buckets = {m: 0 for m in months}
samples = []

reader = pd.read_csv(
    SRC,
    parse_dates=["Timestamp"],
    chunksize=100_000
)

for chunk in reader:
    chunk["month"] = chunk["Timestamp"].dt.to_period("M")
    
    for m in months:
        needed = quota - buckets[m]
        if needed <= 0:
            continue
        
        sub = chunk[chunk["month"] == m]
        if sub.empty:
            continue
        
        pick = sub.sample(min(len(sub), needed), random_state=42)
        samples.append(pick)
        buckets[m] += len(pick)
        
    # Як тільки всі місяці досягли квоти — перериваємо
    if all(count >= quota for count in buckets.values()):
        break

# 4) Об’єднуємо і видаляємо технічну колонку
df_out = pd.concat(samples, ignore_index=True).drop(columns="month")

# 5) Запис у CSV
df_out.to_csv(DST, index=False)

# 6) Перевірка
print("Усього рядків у файлі:", len(df_out))
print("Розподіл по місяцях:")
print(pd.Series(buckets))


Місяці в даних: [Period('2024-10', 'M'), Period('2024-11', 'M'), Period('2024-12', 'M')]
Квота на місяць: 70000 записів
Усього рядків у файлі: 210000
Розподіл по місяцях:
2024-10    70000
2024-11    70000
2024-12    70000
Freq: M, dtype: int64


In [7]:
import os
os.chdir("/Users/user/Documents/DataEngineering")
print("Working dir:", os.getcwd())


Working dir: /Users/user/Documents/DataEngineering


In [ ]:
print(os.listdir("SourceDatasets"))
